# Note: Packages listed below must be installed  beforehand (only for the first time)

Run the first cell to install all the packages (remove the # and run the command)

In [ ]:
#pip install -r packages.txt

In [1]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
import chembl_webresource_client
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from Nmr2Graph_May import *


C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\pybel\struct\operations.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Read files


In [16]:
#Orininal Data
#nmrData = pd.read_csv('data/normalized_data/NMR_data_new.csv')
nmrData = pd.read_csv('data/normalized_data/Ligands.csv')
nmrData['Target'] = 'nsp5'

#Similarity Search on ChEMBL #see the other jupyter notebook file
#nmr2chembl = pd.read_excel('data/normalized_data/Protein-Screening-Summary_CHEMBL30_Actives_teams_max08tan.xlsx')
nmr2chembl = pd.read_excel('data/normalized_data/results/chembl_fpa_apr23.xlsx')

#Similarity Search on PubChem #see the other jupyter notebook file
#nmr2pchem = pd.read_excel('data/normalized_data/PubChem_Actives_Literature_NCATS.xlsx',sheet_name='Similar_065')
nmr2pchem = pd.read_excel('data/normalized_data/results/pchem_fpa_apr23.xlsx')

In [15]:
nmr2pchem.head(1)

,Z-ID,NMR_fragm_structure,NMR_fragm_structure (Canonical),CID,Chembl_structure,Chembl_structure (Canonical),Tanimoto,Source,aid,cid
0,Z1430613393,NaN,Cc1ccc(NC(=O)c2cccc(F)c2)cc1O,837703,NaN,Cc1cc(C(C#N)c2ccc(Cl)cc2)c(Cl)cc1NC(=O)c1cc(I)...,0.842105,NCATS,1479148,42574


# You can scroll all the way down to directly create sub-graphs
## Skip the following line of codes and look for "Start here" down below

# Create an emtpy Graph 

In [8]:
nmrGraph = pybel.BELGraph(name='NMRgraph')

# Populate Graph

In [11]:
for i in range(len(nmrData)):
    nmrGraph.add_association(Abundance(namespace='Enamine',name=str(nmrData['PubChem CID'][i])),Protein(namespace='VP', 
    name=nmrData['Target'][i]),citation="CovidNMR",evidence='NMR_data')

In [18]:
for i in range(len(nmr2chembl)):
    nmrGraph.add_association(Abundance(namespace='Enamine',name=str(nmr2chembl['Frag_id'][i])),Pathology(namespace='ChEMBL',name=str(nmr2chembl['Ref_id'][i])),citation='ChEMBL',evidence='Similarity search')
    #nmrGraph.add_association(Pathology(namespace='ChEMBL',name=nmr2chembl['Molecule ChEMBL ID'][i]),MicroRna(namespace='HP',name=nmr2chembl['Target Name'][i]),citation='ChEMBL', evidence = 'Similarity search')

In [20]:
for i in range(len(nmr2pchem)):
    nmrGraph.add_association(Abundance(namespace='Enamine',name=str(nmr2pchem['Frag_id'][i])),Population(namespace='CID',name=str(nmr2pchem['Ref_id'][i])),citation='Pubchem',evidence='Similarity search')
    #nmrGraph.add_association(Pathology(namespace='ChEMBL',name=nmr2pchem['Molecule ChEMBL ID'][i]),MicroRna(namespace='HP',name=nmr2chembl['Target Name'][i]),citation='ChEMBL', evidence = 'Similarity search')

# Visualize Graph

In [21]:
to_jupyter(nmrGraph)

<IPython.core.display.Javascript object>

In [ ]:
nmrGraph.summarize()

In [ ]:
chemList = list(nmr2chembl['Molecule ChEMBL ID'])
uniqChemList = list(set(chemList))
len(uniqChemList)

In [ ]:
%load Nmr2Graph_May.py

In [8]:
mechList = RetMech(uniqChemList)


NameError: name 'RetMech' is not defined

In [ ]:
ActList = RetAct(uniqChemList,0)

In [ ]:
drugIndList = RetDrugInd(uniqChemList)

In [ ]:
protein_ActList = []
for item in ActList:
    for j in range(len(ActList[item])):
        protein_ActList.append(ActList[item][j]['target_chembl_id'])


In [ ]:
len(protein_ActList)

In [ ]:
protein_ActList = set(protein_ActList)
protein_ActList = set(protein_ActList)

In [ ]:
protein_mechList = Ret_chembl_protein(mechList)
len(protein_mechList)

In [ ]:
protein_ActList = Ret_chembl_protein(ActList)
len(protein_ActList)

In [ ]:
protein_All = protein_ActList + protein_mechList
len(protein_All)

In [ ]:
chem2Gene_all = chembl2uniprot(protein_All,0)

In [ ]:
#convert chembl ids to proteins wherever possible
ActList2gene2Path = chembl2gene2path(chem2Gene_all,ActList)

In [ ]:
#convert chembl ids to proteins wherever possible
mechList = chembl2gene2path(chem2Gene_all,mechList)

In [ ]:
createGraph = chem2act_rel_2(ActList2gene2Path,nmrGraph)
createGraph = chem2dis_rel(drugIndList,nmrGraph)
createGraph = chem2moa_rel_2(mechList,nmrGraph)
createGraph = chem2gene2path_rel(chem2Gene_all,nmrGraph)

In [ ]:
# filename='data\\normalized_data\\FinalGraph_BELformat'
# outfile = open(filename,'wb')
# pickle.dump(nmrGraph,outfile)
# outfile.close()

# Start here

In [5]:
infile = open('data/normalized_data/FinalGraph_BELformat','rb')
nmrGraph = pickle.load(infile)
infile.close()

In [ ]:
#Visualize the entire graph 
#No need to run this code if you feel your pc is getting slow 
#Click on this cell -> Click Cell on tabs menu (File, Edit, View...) -> Current Oupputs -> Clear
to_jupyter(nmrGraph)

In [7]:
#generate graph statistics
nmrGraph.summarize()

---------------------  --------
Name                   NMRgraph
Version
Number of Nodes        1121
Number of Namespaces   9
Number of Edges        4084
Number of Annotations  0
Number of Citations    4
Number of Authors      0
Number of Components   2
Number of Warnings     0
Network Density        3.25E-03
---------------------  --------

Type (8)             Count  Example
-----------------  -------  ----------------------------------------------------------
Abundance              283  a(Enamine:Z30272547)
Gene                   241  g(Pathway:"DCC mediated attractive signaling. [P06241-1]")
Population             177  pop(Disease:"HIV Infections")
RNA                    168  r(ChEMBLAssay:CHEMBL1613803)
miRNA                  137  m(HP:CHEMBL1293226)
Pathology               77  path(ChEMBL:CHEMBL1422849)
Protein                 24  p(VP:"nsp10·nsp16")
BiologicalProcess       14  bp(MOA:"D2-like dopamine receptor agonist")

Prefix (9)    Name                        Count  Example
--

# Function to filter graph on selective viral proteins

In [8]:
nsp5_prot = ['_GHM_nsp5','_GS_nsp5','Nsp5 WT']
nsp3_prot = ['nsp3a','nsp3b','nsp3b·GS-441524','nsp3c (SUD-MC)','nsp3c (SUD-N)','nsp3d','nsp3e','nsp3y']
sig_chembl = ['CHEMBL289356','CHEMBL264373','CHEMBL2105450','CHEMBL1380480',
              'CHEMBL226652','CHEMBL243652','CHEMBL566136','CHEMBL1616','CHEMBL1382627']

#sig_chembl = ['CHEMBL4495583','CHEMBL4303812','CHEMBL1382627']

def filter_graph(mainGraph,vprotList):
    nsp_list = []
    for u, v, data in mainGraph.edges(data=True): 
        if u.name in vprotList or v.name in vprotList: 
            nsp_list.append(u)
            nsp_list.append(v)
        
    for u, v, data in mainGraph.edges(data=True): 
        if 'Similarity' not in data: 
            continue 
        if data['Similarity'] >= .7 and (u in nsp_list or v in nsp_list): 
            #yield u, v, k, data
            nsp_list.append(u)
            nsp_list.append(v)
            
    #print(nsp_list)
    
    nsp_graph = mainGraph.subgraph(nsp_list)
    #return(to_jupyter(nsp_graph))
    final_nsp = [node for node in nsp_graph.nodes() if isinstance(node,pybel.dsl.Protein) and node.name in vprotList]
    #print(final_nsp)
    for u, v, data in nsp_graph.edges(data=True): 
        if u.namespace in ['Enamine','ChEMBL','CID'] and v.namespace in ['Enamine','ChEMBL','CID']:
            final_nsp.append(u)
            final_nsp.append(v)
    
    #return(nsp_graph.subgraph(final_nsp))
    nsp_graph = nsp_graph.subgraph(final_nsp)
    #chembl_act = []
    nsp_nodes = [node for node in nsp_graph.nodes()]
    for u in nsp_graph.nodes():
        if u.namespace == 'ChEMBL':
            #test = nmrGraph.neighbors(u)
            chem = [n for n in mainGraph.neighbors(u)]
            nsp_nodes.extend(chem)
            
    return(mainGraph.subgraph(nsp_nodes))

    
    

# Sub-graph for nsp5 proteins

In [9]:
nsp5Graph = filter_graph(nmrGraph,nsp5_prot)
to_jupyter(nsp5Graph)

<IPython.core.display.Javascript object>

# Sub-graph for nsp3 proteins

In [10]:
nsp3Graph = filter_graph(nmrGraph,nsp3_prot)
to_jupyter(nsp3Graph)

<IPython.core.display.Javascript object>

In [11]:
sig_chembl_graph = filter_graph(nmrGraph,sig_chembl)
to_jupyter(sig_chembl_graph)

<IPython.core.display.Javascript object>

## Thank You